In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import io
from google.colab import files
from sklearn.model_selection import train_test_split
import warnings; warnings.simplefilter('ignore')
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def import_dataset():
  for _ in range(4):
    uploaded = files.upload()
  income_data = pd.read_csv('income_data.csv')
  user_data = pd.read_csv('user_data.csv')
  task_data = pd.read_csv('task_data.csv')
  assignment_data = pd.read_csv('assignment_data.csv')
  return income_data, user_data, task_data, assignment_data

In [ ]:
income_data, user_data, task_data, assignment_data = import_dataset()

Saving income_data.csv to income_data.csv


Saving task_data.csv to task_data.csv


Saving user_data.csv to user_data.csv


Saving assignment_data.csv to assignment_data.csv


In [ ]:
pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.4 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=71577d2e76b3818a6207e3a8a9ca09a0dc90b2d636a72673bf1bf85befae1817
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence_transformers


In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch

# save model in current directory
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2', device='cpu', cache_folder='./')

In [ ]:
# Clean and preprocess text data
task_data['title'] = task_data['title'].fillna('')
task_data['description'] = task_data['description'].fillna('')
task_data['text'] = task_data['title'] + ' ' + task_data['description']

In [ ]:
# Convert categorical features to numerical features
task_data['mayContainAdultContent'] = task_data['mayContainAdultContent'].astype('category').cat.codes
task_data['lang'] = task_data['lang'].astype('category').cat.codes

In [ ]:
def clean_text(author):
    result = str(author).lower()
    return(result.replace(' ',''))

In [ ]:
task_data['text'] = task_data['text'].apply(clean_text)

In [ ]:
# Create a pivot table with users as rows and tasks as columns
pivot = pd.pivot_table(income_data, index='uid', columns='project_id', values='income', fill_value=0)

In [ ]:
corpus_embedding = model.encode(corpus, convert_to_tensor=True)

top_k = min(5, len(corpus))

for query in queries:
    query_embedding = model.encode(query, convert_to_tensor=True)

    cos_scores = util.cos_sim(query_embedding, corpus_embedding)[0]
    top_results = torch.topk(cos_scores, k=top_k)
    print("Query:", query)
    print("---------------------------")
    for score, idx in zip(top_results[0], top_results[1]):
        print(f'{round(score.item(), 3)} | {titles[idx]}')

In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


# Load the pre-trained multilingual model from Sentence Transformers
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

# Clean the text data in task_data
task_data['description'] = task_data['description'].fillna('')
task_data['title'] = task_data['title'].fillna('')
task_data['text'] = task_data['title'] + ' ' + task_data['description']
task_data['text'] = task_data['text'].str.replace('[^a-zA-Z0-9\s]', '')

# Convert categorical features to numeric
task_data['requesterID'] = task_data['requesterID'].astype('category').cat.codes

# Compute the sentence embeddings using the pre-trained multilingual model
sentences = task_data['text'].tolist()
sentence_embeddings = model.encode(sentences)

# Compute the cosine similarity matrix
cosine_sim = cosine_similarity(sentence_embeddings, sentence_embeddings)

# Create a dictionary to map task IDs to their indices in the task_data DataFrame
id_to_index = dict(zip(task_data['project_id'], range(len(task_data))))

# Define the function to get top-K recommendations for a user
def get_top_k_recommendations(user_id, k=5):
    # Get the project IDs completed by the user
    completed_projects = set(income_data[income_data['uid'] == user_id]['project_id'])
    
    # Get the project IDs assigned to the user
    assigned_projects = set(assignment_data[assignment_data['uid'] == user_id]['project_id'])
    
    # Combine the completed and assigned project IDs
    selected_projects = completed_projects.union(assigned_projects)
    
    # Get the indices of the selected projects
    selected_indices = [id_to_index[project_id] for project_id in selected_projects if project_id in id_to_index]
    
    # Compute the similarity scores between the selected projects and all the other projects
    scores = cosine_sim[selected_indices].sum(axis=0)
    
    # Sort the scores and get the top-K recommendations
    top_k = np.argsort(-scores)[:k]
    
    # Get the project IDs corresponding to the top-K indices
    top_k_projects = [task_data.iloc[index]['project_id'] for index in top_k]

    # Get the titles of the projects
    titles_k =  [task_data.iloc[index]['title'] for index in top_k]
    
    return top_k_projects, titles_k

def ndcg_at_k(recommended_items, relevant_items, k):
    dcg = 0.0
    idcg = 0.0
    if len(recommended_items) < k:
        k = len(recommended_items)
    for i in range(k):
        item = recommended_items[i]
        if item in relevant_items:
            dcg += 1.0 / np.log2(i + 2)
    for i in range(min(len(relevant_items), k)):
        idcg += 1.0 / np.log2(i + 2)
    return dcg / idcg



def evaluate_model(k=5):
    # Initialize the scores
    map_k = 0.0
    ndcg_k = 0.0
    num_users = 0
    
    # Loop over all the users
    for user_id in user_data['uid']:
        # Get the ground truth and the predicted top-K recommendations
        ground_truth = set(income_data[income_data['uid'] == user_id]['project_id'])
        p_ids, titles_k = get_top_k_recommendations(user_id, k=k)
        predicted = set(p_ids)
        # predicted = set(get_top_k_recommendations(user_id, k=k))
        
        # Compute the MAP@K and NDCG@K for the user
        num_hits = len(ground_truth.intersection(predicted))
        if num_hits > 0:
            precision = float(num_hits) / float(k)
            recall = float(num_hits) / float(len(ground_truth))
            map_k += precision
            ndcg_k += ndcg_at_k(list(predicted), list(ground_truth), k=k)
            num_users += 1
            
    # Normalize the scores by the number of users
    map_k /= float(num_users)
    ndcg_k /= float(num_users)
    
    return map_k, ndcg_k

In [ ]:
evaluate_model()

(0.46666666666666673, 0.5156223656766757)

In [ ]:
user_data['uid'].unique()

array([1316040758, 1429235843, 1430094402, 1458049517, 1490584733,
       1648608319, 1674749132, 1675921760, 1684019585, 1699065816,
       1703905621, 1706440161, 1709755371, 1710096477, 1713698815,
       1714073899, 1716528528, 1718146027, 1718416044, 1725201772,
       1725259597])

In [ ]:
get_top_k_recommendations(1316040758, k=10)

[81077, 43625, 71859, 80983, 74976, 107310, 107310, 90888, 80999, 91088]

In [ ]:
def get_top_k_recommendations(user_id, k=5):
    # Get the project IDs completed by the user
    completed_projects = set(income_data[income_data['uid'] == user_id]['project_id'])
    
    # Get the project IDs assigned to the user
    assigned_projects = set(assignment_data[assignment_data['uid'] == user_id]['project_id'])
    
    # Combine the completed and assigned project IDs
    selected_projects = completed_projects.union(assigned_projects)
    
    # Get the indices of the selected projects
    selected_indices = [id_to_index[project_id] for project_id in selected_projects if project_id in id_to_index]
    
    # Compute the similarity scores between the selected projects and all the other projects
    scores = cosine_sim[selected_indices].sum(axis=0)
    
    # Sort the scores and get the top-K recommendations
    top_k = np.argsort(-scores)[:k]
    
    # Get the project IDs corresponding to the top-K indices
    top_k_projects = [task_data.iloc[index]['project_id'] for index in top_k]

    # Get the titles of the projects
    titles_k =  [task_data.iloc[index]['title'] for index in top_k]
    
    return top_k_projects, titles_k

In [ ]:
p_ids, titles_k = get_top_k_recommendations(1316040758, 5)
predicted = set(p_ids)
print(predicted)
print(titles_k)

{74976, 80999, 90888, 43625, 107310, 91088, 71859, 81077, 80983}
['Product Image Validation V1', 'Validation of English Ad Description', 'Definitive Image Labeling - EN', 'Product Price Validation V1', 'MMRepresentativeImageV2', '🌼 Is the product relevant to the query?', '🌼 Is the product relevant to the query?', "Stimmt der Screenshot des Coupons mit den Informationen des angezeigten Coupons überein? [DE: Does coupon screenshot match the provided coupon's information?] ", 'Product Title Validation V1', ' La capture d’écran du coupon correspond-elle aux informations du coupon fourni ?']


In [ ]:
from tabulate import tabulate

def display_recommendations(k):

    for i in user_data['uid'].unique():

      list1, list2 = get_top_k_recommendations(i, k)
      # Zip the two lists to create a list of tuples
      zipped_lists = list(zip(list1, list2))

      print("User ID: ", i)
      # Display the zipped lists as a table
      print(tabulate(zipped_lists, headers=['Porject ID', 'Title']))

      print('--------------------------------')

In [ ]:
display_recommendations(5)

User ID:  1316040758
  Porject ID  Title
------------  ------------------------------------
       81077  Product Image Validation V1
       43625  Validation of English Ad Description
       71859  Definitive Image Labeling - EN
       80983  Product Price Validation V1
       74976  MMRepresentativeImageV2
--------------------------------
User ID:  1429235843
  Porject ID  Title
------------  ----------------------------------------
       81077  Product Image Validation V1
       80983  Product Price Validation V1
       80999  Product Title Validation V1
      107310  🌼 Is the product relevant to the query?
      107310  🌼 Is the product relevant to the query?
--------------------------------
User ID:  1430094402
  Porject ID  Title
------------  -------------------------------------
      121035  Новостность документа
      102454  Модерация фотографий отельных номеров
       77419  Распознавание таблиц
       23183  Модерация названий организаций
       22597  Модерация описаний 

## HYPER PARAMETER TUNING

In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import ParameterGrid


# Load the pre-trained multilingual model from Sentence Transformers
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

# Clean the text data in task_data
task_data['description'] = task_data['description'].fillna('')
task_data['title'] = task_data['title'].fillna('')
task_data['text'] = task_data['title'] + ' ' + task_data['description']
task_data['text'] = task_data['text'].str.replace('[^a-zA-Z0-9\s]', '')

# Convert categorical features to numeric
task_data['requesterID'] = task_data['requesterID'].astype('category').cat.codes

# Compute the sentence embeddings using the pre-trained multilingual model
sentences = task_data['text'].tolist()
sentence_embeddings = model.encode(sentences)

# Define the function to get top-K recommendations for a user
def get_top_k_recommendations(user_id, k=5, cosine_sim=cosine_sim, id_to_index=id_to_index, task_data=task_data, income_data=income_data, assignment_data=assignment_data):
    # Get the project IDs completed by the user
    completed_projects = set(income_data[income_data['uid'] == user_id]['project_id'])
    
    # Get the project IDs assigned to the user
    assigned_projects = set(assignment_data[assignment_data['uid'] == user_id]['project_id'])
    
    # Combine the completed and assigned project IDs
    selected_projects = completed_projects.union(assigned_projects)
    
    # Get the indices of the selected projects
    selected_indices = [id_to_index[project_id] for project_id in selected_projects if project_id in id_to_index]
    
    # Compute the similarity scores between the selected projects and all the other projects
    scores = cosine_sim[selected_indices].sum(axis=0)
    
    # Sort the scores and get the top-K recommendations
    top_k = np.argsort(-scores)[:k]
    
    # Get the project IDs corresponding to the top-K indices
    top_k_projects = [task_data.iloc[index]['project_id'] for index in top_k]

    # Get the titles of the projects
    titles_k =  [task_data.iloc[index]['title'] for index in top_k]
    
    return top_k_projects, titles_k

def ndcg_at_k(recommended_items, relevant_items, k):
    dcg = 0.0
    idcg = 0.0
    if len(recommended_items) < k:
        k = len(recommended_items)
    for i in range(k):
        item = recommended_items[i]
        if item in relevant_items:
            dcg += 1.0 / np.log2(i + 2)
    for i in range(min(len(relevant_items), k)):
        idcg += 1.0 / np.log2(i + 2)
    return dcg / idcg


def evaluate_model(params, k=5):
    # Initialize the scores
    map_k = 0.0
    ndcg_k = 0.0
    num_users = 0
    
    # Loop over all the users
    for user_id in user_data['uid']:
        # Get the ground truth and the predicted top-K recommendations
        ground_truth = set(income_data[income_data['uid'] == user_id]['project_id'])
        p_ids, titles_k = get_top_k_recommendations(user_id, k=k)
        predicted = set(p_ids)
        # predicted = set(get_top_k_recommendations(user_id, k=k))
        
        # Compute the MAP@K and NDCG@K for the user
        num_hits = len(ground_truth.intersection(predicted))
        if num_hits > 0:
            precision = float(num_hits) / float(k)
            recall = float(num_hits) / float(len(ground_truth))
            map_k += precision
            ndcg_k += ndcg_at_k(list(predicted), list(ground_truth), k=k)
            num_users += 1
            
    # Normalize the scores by the number of users
    map_k /= float(num_users)
    ndcg_k /= float(num_users)
    
    return map_k, ndcg_k

In [ ]:
params = {
    'model': ['paraphrase-multilingual-mpnet-base-v2', 'paraphrase-xlm-r-multilingual-v1'],
    'k': [5, 10, 20],
    'batch_size': [16, 32, 64],
    'epochs': [1, 2, 3],
}

In [ ]:
evaluate_model(params)

(0.46666666666666673, 0.5156223656766757)